# Clase de Machine Learning Aplicada a la Minería
## Introducción
En esta clase, vamos a trabajar con un dataset específico para realizar las siguientes tareas:
1. **Manejo de valores nulos.**
2. **Tratamiento de outliers.**
3. **Normalización y escalado de datos.**

Este proceso es fundamental para preparar los datos antes de aplicar cualquier modelo de Machine Learning, especialmente en un contexto de minería de datos donde la calidad de los datos es crítica.

### Paso 1: Importación de Bibliotecas y Carga de Datos
Primero, importaremos las bibliotecas necesarias y cargaremos el dataset.

In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Cargar el dataset
data = pd.read_csv('data_clase.csv')


# Mostrar las primeras filas del dataset
data.head()

,id,Turno Carga,equipo_acarreo,operador_acarreo,material,flota_carguio,equipo_carguio,duracion_llegada,duracion_acarreo,duracion_descarga,duracion_viajando_vacio,tonelaje,efh_cargado_km,efh_vacio_km,distancia_cargado_km,distancia_vacia_km,velocidad_cargado,velocidad_vacio
0,1104116,A,Equipo_acarreo 1,Operador 1,Mineral Sulfuro,Flota_carguio 1,Equipo_carguio 1,271,120,48,124.2,305.0,1.00,1.24,0.66,0.61,20.22,18.24
1,1104117,A,Equipo_acarreo 2,Operador 2,Mineral Óxido,Flota_carguio 2,Equipo_carguio 2,158,338,50,271.8,275.0,2.82,2.72,2.23,2.30,23.71,26.32
2,1104118,A,Equipo_acarreo 3,Operador 3,Mineral Sulfuro,Flota_carguio 2,Equipo_carguio 3,530,982,34,568.2,279.0,8.18,5.68,4.07,4.37,14.67,32.39
3,1104119,A,Equipo_acarreo 4,Operador 4,Desmonte,Flota_carguio 1,Equipo_carguio 4,6,216,50,448.2,273.0,1.80,4.48,1.28,3.28,20.51,25.73
4,1104120,A,Equipo_acarreo 5,Operador 5,Mineral Óxido,Flota_carguio 2,Equipo_carguio 2,4,340,34,394.8,298.0,2.83,3.95,2.25,2.30,23.95,29.35


### Paso 2: Análisis Exploratorio de Datos
Vamos a realizar un análisis preliminar para entender la estructura de los datos, identificar los tipos de variables y detectar valores nulos.

In [23]:
# Información general del dataset
data.info()

# Resumen estadístico de las variables numéricas
data.describe()

# Detección de valores nulos
null_values = data.isnull().sum()
null_values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38376 entries, 0 to 38375
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       38376 non-null  int64  
 1   Turno Carga              38376 non-null  object 
 2   equipo_acarreo           38376 non-null  object 
 3   operador_acarreo         38376 non-null  object 
 4   material                 38376 non-null  object 
 5   flota_carguio            38376 non-null  object 
 6   equipo_carguio           38376 non-null  object 
 7   duracion_llegada         38376 non-null  int64  
 8   duracion_acarreo         38376 non-null  int64  
 9   duracion_descarga        38376 non-null  int64  
 10  duracion_viajando_vacio  38376 non-null  float64
 11  tonelaje                 38368 non-null  float64
 12  efh_cargado_km           38376 non-null  float64
 13  efh_vacio_km             38376 non-null  float64
 14  distancia_cargado_km  

id                           0
Turno Carga                  0
equipo_acarreo               0
operador_acarreo             0
material                     0
flota_carguio                0
equipo_carguio               0
duracion_llegada             0
duracion_acarreo             0
duracion_descarga            0
duracion_viajando_vacio      0
tonelaje                     8
efh_cargado_km               0
efh_vacio_km                 0
distancia_cargado_km       454
distancia_vacia_km          59
velocidad_cargado           21
velocidad_vacio             30
dtype: int64

### Paso 3: Manejo de Valores Nulos
Dependiendo del tipo de variable (categórica o numérica), aplicaremos diferentes técnicas para manejar los valores nulos.

In [24]:
# Imputación de valores nulos para variables numéricas
data_numeric = data.select_dtypes(include=[np.number])

# Imputación con la media
data_numeric = data_numeric.fillna(data_numeric.mean())

# Imputación de valores nulos para variables categóricas
data_categorical = data.select_dtypes(exclude=[np.number])

# Imputación con el modo (valor más frecuente)
data_categorical = data_categorical.apply(lambda x: x.fillna(x.mode()[0]))

# Reemplazamos las columnas originales con las imputadas
data.update(data_numeric)
data.update(data_categorical)

# Verificamos que no queden valores nulos
data.isnull().sum()

id                         0
Turno Carga                0
equipo_acarreo             0
operador_acarreo           0
material                   0
flota_carguio              0
equipo_carguio             0
duracion_llegada           0
duracion_acarreo           0
duracion_descarga          0
duracion_viajando_vacio    0
tonelaje                   0
efh_cargado_km             0
efh_vacio_km               0
distancia_cargado_km       0
distancia_vacia_km         0
velocidad_cargado          0
velocidad_vacio            0
dtype: int64

### Paso 4: Tratamiento de Outliers
Identificaremos y trataremos los outliers en las variables numéricas.

## Qué es el IQR

El IQR, o **Rango Intercuartílico** (Interquartile Range en inglés), es una medida estadística utilizada para describir la dispersión o variabilidad en un conjunto de datos. Específicamente, el IQR mide la distancia entre el primer cuartil (Q1) y el tercer cuartil (Q3), y se calcula como la diferencia entre estos dos valores:

IQR = Q3 - Q1

Donde:
- **Q1** es el primer cuartil, que representa el valor por debajo del cual se encuentra el 25% de los datos.
- **Q3** es el tercer cuartil, que representa el valor por debajo del cual se encuentra el 75% de los datos.

## Sentido Estadístico del IQR

El IQR es especialmente útil en la estadística descriptiva porque:
1. **Robustez**: A diferencia del rango total (que toma en cuenta el valor máximo y el valor mínimo), el IQR no se ve afectado por los valores atípicos (outliers), lo que lo convierte en una medida más robusta para describir la dispersión central de los datos.
2. **Distribución**: El IQR permite entender mejor la distribución de los datos al enfocarse en la mitad central de estos, proporcionando una idea de la "concentración" de los valores sin considerar los extremos.
3. **Identificación de Outliers**: El IQR también se utiliza en la detección de valores atípicos. Comúnmente, se considera que un dato es un outlier si se encuentra por debajo de \( Q1 - 1.5 \times \text{IQR} \) o por encima de \( Q3 + 1.5 \times \text{IQR} \).

En resumen, el IQR es una herramienta clave para entender la dispersión de los datos, especialmente cuando se desea evitar el impacto de valores extremos que podrían distorsionar la percepción de la variabilidad del conjunto de datos.


In [25]:
# Función para detectar outliers usando el rango intercuartílico (IQR)
def detect_outliers_iqr(df):
    outliers_indices = []
    for col in df.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))].index
        outliers_indices.extend(outliers)
    return list(set(outliers_indices))

# Detectar outliers en el dataset numérico
outliers = detect_outliers_iqr(data_numeric)

# Tratamiento de outliers: se pueden eliminar o ajustar
# En este caso, eliminaremos los outliers
data_cleaned = data.drop(index=outliers)

# Verificamos el resultado
data_cleaned.describe()

,id,duracion_llegada,duracion_acarreo,duracion_descarga,duracion_viajando_vacio,tonelaje,efh_cargado_km,efh_vacio_km,distancia_cargado_km,distancia_vacia_km,velocidad_cargado,velocidad_vacio
count,2.965000e+04,29650.000000,29650.000000,29650.000000,29650.000000,29650.000000,29650.000000,29650.000000,29650.000000,29650.000000,29650.000000,29650.000000
mean,1.123545e+06,78.215582,545.126105,39.867589,384.441936,284.641754,4.518385,3.844433,2.731979,2.776311,20.865620,29.759955
std,1.127101e+04,104.014936,296.114191,7.065733,165.237472,9.301655,2.430129,1.652383,1.022948,1.025191,6.143554,3.978543
min,1.104116e+06,0.000000,0.000000,21.000000,0.000000,259.000000,0.020000,0.000000,0.000000,0.000000,3.330000,17.520000
25%,1.113872e+06,6.000000,242.000000,36.000000,238.200000,278.000000,2.020000,2.380000,1.740000,1.770000,15.760000,27.150000
50%,1.123422e+06,22.000000,648.000000,40.000000,379.800000,285.000000,5.370000,3.800000,3.100000,3.120000,18.760000,29.860000
75%,1.133437e+06,122.000000,796.000000,44.000000,475.200000,291.000000,6.620000,4.750000,3.530000,3.540000,25.860000,32.510000
max,1.142951e+06,418.000000,1620.000000,61.000000,900.000000,310.000000,12.700000,9.000000,6.250000,6.230000,39.430000,40.910000


In [26]:
data_cleaned.to_csv('data_cleaned.csv', index=False)

### Paso 5: Normalización y Escalado de Datos
Finalmente, normalizaremos y escalaremos las variables numéricas para que estén en una misma escala.

In [27]:
# # Normalización usando MinMaxScaler
# scaler = MinMaxScaler()
# data_scaled = pd.DataFrame(scaler.fit_transform(data_cleaned.select_dtypes(include=[np.number])),
#                            columns=data_cleaned.select_dtypes(include=[np.number]).columns)

# # Actualizamos el dataset original con los datos escalados
# data_cleaned.update(data_scaled)

# # Verificamos los datos escalados
# data_cleaned.describe()

### Conclusión
Al finalizar estos pasos, habremos limpiado y preparado los datos, dejándolos listos para aplicar algoritmos de Machine Learning. Cada uno de estos pasos es crucial para asegurar la calidad de los datos y, por lo tanto, la efectividad de los modelos que se construyan posteriormente.

In [28]:
data_cleaned.shape

(29650, 18)